# TEST

# Google Colab Free GPU

https://colab.research.google.com/notebooks/welcome.ipynb

# Starting importing packages

In [8]:
import tensorflow as tf
import numpy as np
import pandas as pd

# Split arrays or matrices into random train and test subsets
from sklearn.model_selection import train_test_split

C:\Users\1511 MXTI\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


# Set pre-process according to input data

In [9]:
def preprocess_x(df_x):
    """Set pre process according to input data.

    Args:
    input_features: Ten entries, from x0 to x9 and from y0 to y9.
    Returns:
    A set of feature columns.
    """
    return df_x[['x0','y0','x1','y1','x2','y2','x3','y3','x4','y4','x5','y5','x6','y6','x7','y7','x8','y8','x9','y9']].values

# Define function for data output

In [0]:
def preprocess_y(df_y):
    """Define function for data output.
    
    Obs. delete the id is necessary.
    Args:
    input_features: The names of the numerical input features to use.
    Returns:
    A set of feature columns.
    """
    return df_y[["slope", "intercept"]].values

# Importing the 3 worksheets related to the project: test_100k.csv, train_100k.csv and Saving train_100k.truth.csv

In [5]:
from google.colab import files
def getLocalFiles():
    _files = files.upload()
    if len(_files) >0:
       for k,v in _files.items():
         open(k,'wb').write(v)
getLocalFiles()

Saving test_100k.csv to test_100k (2).csv


In [6]:
from google.colab import files
def getLocalFiles():
    _files = files.upload()
    if len(_files) >0:
       for k,v in _files.items():
         open(k,'wb').write(v)
getLocalFiles()

Saving train_100k.csv to train_100k (1).csv


In [7]:
from google.colab import files
def getLocalFiles():
    _files = files.upload()
    if len(_files) >0:
       for k,v in _files.items():
         open(k,'wb').write(v)
getLocalFiles()

Saving train_100k.truth.csv to train_100k.truth (1).csv


# Load train datasets, process columns and split by train/test data 90%/10%

In [0]:
x_data = pd.read_csv("train_100k.csv")
x_data = preprocess_x(x_data)

y_data = pd.read_csv("train_100k.truth.csv")
y_data =  preprocess_y(y_data)
    """Define the set into 10 groups (splits) for 10x of which 9 will be used for training and 1 will be used for testing.
    
    test_size - defines the percentage of events assigned to the test set.
    random_state - controls which points are included in the training set and that are used to test.
    """
x_data_train, x_data_test, y_data_train, y_data_test = train_test_split(x_data,
                                                                        y_data,
                                                                        test_size = 0.1,
                                                                        random_state = 100)

# Load test dataset and process columns

In [0]:
x_data_eval = pd.read_csv("test_100k.csv")
x_data_eval = preprocess_x(x_data_eval)

# Define the parameters for training the neural network

In [0]:
    """Define the parameters for training the neural network.
    
    learning_rate - hyper-parameters to tune for training deep neural networks.
    training_epochs - number of times the whole training data is shown to the network while training (iterations).
    batch_size - number of sub samples given to the network after which parameter update happens.
    display_step - number of checkpoints.
    train_data_size - 90% will be used for training.
    test_data_size - 10% will be used for test.
    n_hidden - number of neurons used into hidden layers.
    n_input - number of nodes.
    n_output - number of nodes.
    """

learning_rate = 0.0001
training_epochs = 5000
batch_size = 1000
display_step = 100

train_data_size = 90000
test_data_size = 10000
data_size = train_data_size + test_data_size

n_hidden = 1000

n_input = 20
n_output = 2

# TensorFlow graph input

In [0]:
# tf Graph input
x = tf.placeholder("float", [None, n_input])
y = tf.placeholder("float", [None, n_output])

# dropout (keep probability)
dropout_keep_prob = tf.placeholder(tf.float32)

# Randomly initialize the weights and biases considering their proper dimensions

In [0]:
weights = {
    'h1': tf.Variable(tf.random_normal([n_input, n_hidden],stddev=0.1)),
    'h2': tf.Variable(tf.random_normal([n_hidden, n_hidden],stddev=0.1)),
    'h3': tf.Variable(tf.random_normal([n_hidden, n_hidden],stddev=0.1)),
    'h4': tf.Variable(tf.random_normal([n_hidden, n_hidden],stddev=0.1)),
    'h5': tf.Variable(tf.random_normal([n_hidden, n_hidden],stddev=0.1)),
    'out': tf.Variable(tf.random_normal([n_hidden, n_output],stddev=0.1))
}
biases = {
    'b1': tf.Variable(tf.random_normal([n_hidden])),
    'b2': tf.Variable(tf.random_normal([n_hidden])),
    'b3': tf.Variable(tf.random_normal([n_hidden])),
    'b4': tf.Variable(tf.random_normal([n_hidden])),
    'b5': tf.Variable(tf.random_normal([n_hidden])),
    'out': tf.Variable(tf.random_normal([n_output]))
}

# Connect the Multilayer Perceptron

In [0]:
def make_network(x, weights, biases, dropout_keep_prob):
    """Defined the Tanh function:
    
    * mainly used classification between two classes.
    * one of those functions that can provide nonlinearity such that the entire neural network can 
    be used for universal approximation.
    * its value is restricted between -1 and 1.
    * its gradient is less computational expensive.
    """
    # hidden fully connected layer with tanh activation
    layer_1 = tf.add(tf.matmul(x, weights['h1']), biases['b1'])
    layer_1 = tf.nn.dropout(tf.nn.tanh(layer_1), dropout_keep_prob)

    layer_2 = tf.add(tf.matmul(layer_1, weights['h2']), biases['b2'])
    layer_2 = tf.nn.dropout(tf.nn.tanh(layer_2), dropout_keep_prob)

    layer_3 = tf.add(tf.matmul(layer_2, weights['h3']), biases['b3'])
    layer_3 = tf.nn.dropout(tf.nn.tanh(layer_3), dropout_keep_prob)
    
    layer_4 = tf.add(tf.matmul(layer_3, weights['h4']), biases['b4'])
    layer_4 = tf.nn.dropout(tf.nn.tanh(layer_4), dropout_keep_prob)
    
    layer_5 = tf.add(tf.matmul(layer_4, weights['h5']), biases['b5'])
    layer_5 = tf.nn.tanh(layer_5)
    
    # output fully connected layer for each class
    out_layer = tf.matmul(layer_5, weights['out']) + biases['out']
    return out_layer

# Set up calculations

In [0]:
# run feed forward
y_pred = make_network(x, weights, biases, dropout_keep_prob)

# calculate MSE - computes the mean of elements across dimensions of a tensor
slope_cost = tf.reduce_mean(tf.square(y_pred[:,0]-y[:,0]))

# calculate MAE - computes the mean of elements across dimensions of a tensor
intercept_cost = tf.reduce_mean(tf.abs(y_pred[:,1]-y[:,1]))

# run backward propagation to adjust cost - optimizer that implements the Adam algorithm
# adam work well with multi-layer Neural Networks
# it is reputed to work well for both sparse matrices and noisy data
slope_optimizer =  tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(slope_cost)
intercept_optimizer =  tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(intercept_cost)

# Session object encapsulation

In [15]:
# Create batches to train in more small steps
# Create a default in-process session.
    """Since a tf.Session owns physical resources (such as GPUs and network connections), 
    it is typically used as a context manager (in a with block) that automatically closes 
    the session when you exit the block.
    
    Args:
    Each TensorFlow Session will try to use ALL available resources by default. 
    It encapsulates the environment in which Operation objects are executed and evaluated.
    It allocates resources (on one or more machines) for that and holds the actual values of intermediate 
    results and variables.
    Use the context manager in Python for the best practice.
    Within a session, we initialise variables for each epoch and for each batch, 
    running optimisation op (backprop) and cost op (to get loss value), computing average cost for batch
    and displaying average cost for this epoch.

    Returns:
    Using the optimizer to train the model we reduce the cost. 
    sess.run accepts list of graph elements and returns at the final of the code, 
    the files submission.train_100k.csv and submission.test_100k.csv. 
    """
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    total_batch = int(train_data_size / batch_size)
    
    # training cycle
    for epoch in range(training_epochs):
        avg_cost = 0
        avg_cost2 = 0
        
        # loop over all batches
        for i in range(total_batch):
            batch_x, batch_y = x_data_train[i * batch_size:min(i * batch_size + batch_size, train_data_size), :], \
                             y_data_train[i * batch_size:min(i * batch_size + batch_size, train_data_size), :]
            # Train batch
            # Run optimization op (backprop) and cost op (to get loss value)
            sess.run([slope_optimizer,  intercept_optimizer], feed_dict={x: batch_x, y: batch_y, dropout_keep_prob: 0.9})
            
            # Calculate cost
            slope_c, intercept_c = sess.run([slope_cost, intercept_cost], feed_dict={x: batch_x, y: batch_y, dropout_keep_prob: 1.})
            
            # Compute average loss
            avg_cost += slope_c / total_batch
            avg_cost2 += intercept_c / total_batch
        
        # Display logs per epoch step
        if (epoch+1) % display_step == 0:
            print('Epoch:', (epoch + 1), 'Slope MSE cost=', '{:.9f}'.format(avg_cost), 'Intercept MAE cost=', '{:.9f}'.format(avg_cost2))
    
    # Generate train result 
    r = sess.run(y_pred, feed_dict = {x: x_data, dropout_keep_prob: 1.})
    r = pd.DataFrame(r, columns=["slope", "intercept"])
    r.to_csv("submission.train_100k.csv", index=True, index_label='id')
    
    # Generate test result
    r = sess.run(y_pred, feed_dict = {x: x_data_eval, dropout_keep_prob: 1.})
    r = pd.DataFrame(r, columns=["slope", "intercept"])
    r.to_csv("submission.test_100k.csv", index=True, index_label='id')

Epoch: 100 Slope MSE cost= 0.089810066 Intercept MAE cost= 20.309488191
Epoch: 200 Slope MSE cost= 0.028637709 Intercept MAE cost= 13.299832567
Epoch: 300 Slope MSE cost= 0.015130374 Intercept MAE cost= 9.408265771
Epoch: 400 Slope MSE cost= 0.010196449 Intercept MAE cost= 7.746847608
Epoch: 500 Slope MSE cost= 0.007531849 Intercept MAE cost= 6.616834752
Epoch: 600 Slope MSE cost= 0.006002341 Intercept MAE cost= 6.063143094
Epoch: 700 Slope MSE cost= 0.005080163 Intercept MAE cost= 5.510033210
Epoch: 800 Slope MSE cost= 0.004390351 Intercept MAE cost= 5.097903803
Epoch: 900 Slope MSE cost= 0.003821173 Intercept MAE cost= 4.748608685
Epoch: 1000 Slope MSE cost= 0.003459685 Intercept MAE cost= 4.681830976
Epoch: 1100 Slope MSE cost= 0.003454438 Intercept MAE cost= 4.156887354
Epoch: 1200 Slope MSE cost= 0.002854744 Intercept MAE cost= 3.951280313
Epoch: 1300 Slope MSE cost= 0.002996455 Intercept MAE cost= 3.755739050
Epoch: 1400 Slope MSE cost= 0.002462681 Intercept MAE cost= 3.647110772